In [75]:
import Pkg; Pkg.activate(@__DIR__)
using Pkg; Pkg.instantiate()
using DifferentialEquations, LinearAlgebra, Statistics, Random
using Plots, Distributions, KernelDensity, StatsBase, CSV, DataFrames
using LaTeXStrings, Measures

  Activating project at `~/Documents/Bachelor Thesis Stuff/bachelorsthesis`


### Reaction Kinetics

In [76]:
# === Load Experimental Data ===
df = CSV.read("datasetsMA/nitrogenlim.csv", DataFrame)
dfs = hcat(df.Xa, df.Xi, df.N, df.S, df.FG, df.MA)
u0 = dfs[1, :]
t_exp = df.time
T = maximum(t_exp)
tspan = (0.0, T)
dt = 0.01
tsteps = collect(0.0:dt:T)

# === Define ODE Model ===
function f!(du, u, p, t)
    # Unpack state and parameters
    μmax, KFG, KN, YXa_S, YXi_S, YXa_N, YP_S, ϕ, χacc, μ2max, qsplit_max, Ksuc, qpmax, KIP, KIN, KPFG, KFG2, σxa, σxi, σs, σfg, σp = p
    Xact, Xinact, N, Suc, FruGlu, P = u

    # Ensure non-negative values
    ϵ = 1e-8  # Small positive value to avoid division by zero
    Xact_safe = max(Xact, ϵ)
    Xtot_safe = max(Xact + Xinact, ϵ)
    FruGlu_safe = max(FruGlu, ϵ)
    Suc_safe = max(Suc, ϵ)

    # Algebraic equations
    Xtot = Xact + Xinact
    N_int = 0.08 * N
    ratio = Xinact / Xact_safe
    expo_term = (ratio - ϕ) / χacc

    μ = μmax * FruGlu_safe / (FruGlu_safe + KFG + ϵ) * (N / (N+ KN + ϵ))
    μ2 = μ2max * FruGlu_safe / (FruGlu_safe + KFG2 + ϵ) * (1 - exp(expo_term)) * KIN / (KIN + N + ϵ)
    qsplit = qsplit_max * Suc_safe / (Suc_safe + Ksuc + ϵ)
    qp = qpmax * FruGlu_safe / (FruGlu_safe + KPFG + ϵ) *
         (KIP / (KIP + N_int / Xtot_safe + ϵ)) * KIN / (KIN + N + ϵ)

    du[1] = μ * Xact
    du[2] = μ2 * Xact
    du[3] = - (μ / YXa_N) * Xact
    du[4] = - qsplit * Xact
    du[5] = (qsplit - μ / YXa_S - μ2 / YXi_S - qp / YP_S) * Xact
    du[6] = qp * Xact
end

# === Define Noise Function ===
function noise!(du, u, p, t)
    Xact, Xinact, N, Suc, FruGlu, P = u
    μmax, KFG, KN, YXa_S, YXi_S, YXa_N, YP_S, ϕ, χacc, μ2max, qsplit_max, Ksuc, qpmax, KIP, KIN, KPFG, KFG2, σxa, σxi, σs, σfg, σp = p
    du[1] = σxa * FruGlu / (FruGlu + (KFG)) * Xact
    du[2] = σxi * FruGlu / (FruGlu + (KFG2)) * Xinact
    # du[3] = (σxa / YXa_N) * FruGlu / (FruGlu + (KFG)) * Xact
    # du[4] = σs * Suc / (Suc + Ksuc) * Xact
    du[5] = σfg * FruGlu / (FruGlu + Ksuc) * (Xact + Xinact)
    du[6] = σp * FruGlu / (FruGlu + (KPFG)) * Xact
end

# === Parameters ===
params = [
    0.125,  # 1. μmax
    0.147,  # 2. KFG
    3.8e-5,  # 3. KN
    0.531,  # 4. YXa_S
    0.799,  # 5. YXi_S
    9.428,  # 6. YXa_N
    0.508,  # 7. YP_S
    1.56,  # 7. ϕ
    0.3,  # 8. χacc
    0.125,  # 9. μ2max
    1.985,  # 10. qsplit_max
    0.00321,  # 11. Ksuc
    0.095,  # 12. qpmax
    1.5,  # 13. KIP
    1.5e-3,  # 14. KIN
    0.0175,  # 15. KPFG
    3.277,  # 16. KFG2
    0.05,   # 17. σxa
    0.05,   # 18. σxi
    0.05,   # 19. σn
    0.05,   # 20. σs
    0.05,   # 21. σfg
    0.05    # 22. σp
]

# === Solve the ODE ===
odeprob = ODEProblem(f!, u0, tspan, params)
odesol = solve(odeprob, Rosenbrock23(), saveat=tsteps, abstol=1e-8, reltol=1e-6)

# === Extract ODE solution ===
Xa_ode = odesol[1, :]
Xi_ode = odesol[2, :]
N_ode  = odesol[3, :]
Suc_ode = odesol[4, :]
FG_ode = odesol[5, :]
P_ode = odesol[6, :]

3984-element Vector{Float64}:
  0.0
  3.3758095455225526e-6
  6.756997945618739e-6
  1.014356910270506e-5
  1.3535523125115027e-5
  1.693289154482288e-5
  2.0335677104137642e-5
  2.3743873682775076e-5
  2.7157481100496607e-5
  3.057656135680802e-5
  ⋮
 21.94255229450724
 21.942552578308216
 21.94255286210919
 21.94255314591016
 21.94255342971113
 21.9425537135121
 21.94255399731307
 21.94255428111404
 21.94255456491501

### Monte Carlo Simulations

In [77]:
# === Define the non-negative callback and the SDE Problem ===
function project!(integrator)
    integrator.u .= max.(integrator.u, 0.0)
end
proj_cb = DiscreteCallback((u,t,integrator) -> true, project!)

sdeprob = SDEProblem(f!, noise!, u0, tspan, params)

# === Simulate the scenarios ===
M = 1000  # Number of simulations

ensemble_prob = EnsembleProblem(sdeprob, prob_func = (prob, i, repeat) -> remake(prob, u0 = u0))
ensemble_sol = solve(ensemble_prob, EM(), EnsembleThreads(), trajectories=M, dt=dt,
                     saveat=tsteps, callback=proj_cb)

# Extract results
Xact_mat = hcat([sol[1, :] for sol in ensemble_sol]...)
Xinact_mat = hcat([sol[2, :] for sol in ensemble_sol]...)
N_mat = hcat([sol[3, :] for sol in ensemble_sol]...)
Suc_mat = hcat([sol[4, :] for sol in ensemble_sol]...)
FruGlu_mat = hcat([sol[5, :] for sol in ensemble_sol]...)
P_mat = hcat([sol[6, :] for sol in ensemble_sol]...)


11937×1000 Matrix{Float64}:
  0.0          0.0          0.0          …   0.0          0.0
 -0.00738537   0.00135768  -0.00147918      -0.0146004    0.000165136
  0.0          0.00135768   0.0              0.0          0.000165136
  0.0158998    0.0131556   -0.00433268       0.0100337    0.00737574
  0.0158998    0.0131556    0.0              0.0100337    0.00737574
  0.00534337   0.0216311   -0.00418595   …   0.00857914   0.0110006
  0.00534337   0.0216311    0.0              0.00857914   0.0110006
 -0.00827171   0.0269239    0.000587117     -0.00302504  -0.00359796
  0.0          0.0269239    0.000587117      0.0          0.0
 -0.0144311    0.0207348    0.0133811       -0.00852418  -0.000298743
  ⋮                                      ⋱               
 17.6421      20.1533      24.9905          18.887       24.3464
 17.6421      20.1533      24.9856          18.9414      24.3483
 17.6421      20.1533      24.9856       …  18.9414      24.3483
 17.6421      20.1533      24.9856        

### Longstaff-Schwartz Utilities

In [89]:
# Laguerre basis functions up to degree 3
function laguerre_design_matrix(y::Vector{Float64}, d::Int)
    Φ = zeros(length(y), d + 1)
    for i in 1:length(y)
        Φ[i,1] = 1.0
        if d >= 1
            Φ[i,2] = 1 - y[i]
        end
        if d >= 2
            Φ[i,3] = 1 - 2*y[i] + 0.5*y[i]^2
        end
        if d >= 3
            Φ[i,4] = 1 - 3*y[i] + 1.5*y[i]^2 - (1/6)*y[i]^3
        end
    end
    return Φ
end

laguerre_design_matrix (generic function with 1 method)